# **Problem Producenta i Konsumenta**

In [ ]:
import threading
import time
import random
import queue

**Wersja z synchronizacją**

In [ ]:
buffer = queue.Queue(10)

# Producent generuje elementy i umieszcza je w buforze
def producer_parallel():
    while True:
        item = random.randint(1, 100)
        print(f"Producent wyprodukował: {item}")
        buffer.put(item)
        time.sleep(1)

# Konsument pobiera elementy z bufora
def consumer_parallel():
    while True:
        item = buffer.get()
        print(f"Konsument skonsumował: {item}")
        time.sleep(random.uniform(1, 3))

producer_thread = threading.Thread(target=producer_parallel)
consumer_thread = threading.Thread(target=consumer_parallel)

producer_thread.start()
consumer_thread.start()

# Ustawianie czasu działania programu
time.sleep(5)

producer_thread.join(1)
consumer_thread.join(1)


**Wersja bez synchronizacji**

In [ ]:
shared_buffer = []

buffer_size = 10

# Producent generuje elementy i umieszcza je w buforze
def producer():
  for i in range(buffer_size):
    item = random.randint(1, 100)
    shared_buffer.append(item)
    print(f"Producent wyprodukował: {item}")
    time.sleep(1)

# Konsument pobiera elementy z bufora
def consumer():
    while True:
      item = shared_buffer.pop(0)
      print(f"Konsument skonsumował: {item}")
      if len(shared_buffer) <= 0:
        break
      time.sleep(random.uniform(1, 3))

producer()
consumer()


Z synchronizacją wątki działają w tym samym czasie, dzięki czemu konsument może konsumować dobra dopiero co wyprodukowane przez producenta.
Bez synchronizacji konsument czeka aż producent skończy wytwarzać i dopiero wtedy konsumuje produkt.

# **Problem pięciu filozofów**

In [2]:
import threading
import time
import random

class Philosopher(threading.Thread):
    def __init__(self, index, left_fork, right_fork):
        threading.Thread.__init__(self)
        self.index = index
        self.left_fork = left_fork
        self.right_fork = right_fork

    # Metoda wywołująca się od razu po włączeniu wątku
    def run(self):
        while True:
            self.think()
            self.get_forks()

    # Czas myślenia filozofa
    def think(self):
        print(f"Filozof {self.index} myśli.")
        time.sleep(random.uniform(1, 3))

    # Próba zdobycia obu widelców
    def get_forks(self):
        fork1, fork2 = self.left_fork, self.right_fork
        while True:
            fork1.acquire(True) # Zdobycie lewego widelca
            locked = fork2.acquire(False) # Zdobycie prawego widelca
            if locked: break # Jeśli filozof ma oba widelce wyjście z pętli
            fork1.release()
            fork1, fork2 = fork2, fork1 # Zamiana widelców
        self.eat()
        fork2.release()
        fork1.release()

    # Czas jedzenia filozofa
    def eat(self):
        print(f"Filozof {self.index} je.")
        time.sleep(random.uniform(1,3))
        print(f"Filozof {self.index} skończył jeść.")

# Inicjalizacja widelców
forks = [threading.Lock() for n in range(5)]
# Inicjalizacja filozofów
philosophers = [Philosopher(i, forks[i % 5], forks[(i + 1) % 5]) for i in range(5)]

for p in philosophers:
    p.start()


Filozof 0 myśli.
Filozof 1 myśli.
Filozof 2 myśli.
Filozof 3 myśli.
Filozof 4 myśli.

**Brak Wzajemnego Wykluczania**

Filozofowie nie używają blokad do uzyskiwania widelców, co może prowadzić do sytuacji, w której dwóch filozofów jednocześnie używa tego samego widelca.

**Niewywłaszczalność**

Filozofowie nie zwalniają widelców, dopóki nie skończą jeść, co może prowadzić do zakleszczenia.

**Blokada**

Filozofowie próbują uzyskać widelce w określonej kolejności i nie zwalniają ich, jeśli nie uzyskają drugiego, co prowadzi do zakleszczenia.

**Zagłodzenie**

Niektórzy filozofowie mogą być nieustannie blokowani przez innych, którzy ciągle zatrzymują widelce.

# **Symulacja kolejki w sklepie**

In [ ]:
import time
import threading
import queue
import random

# Klasa klienta
class Customer(threading.Thread):
    def __init__(self, id):
        threading.Thread.__init__(self)
        self.id = id

    # Przybycie klienta do sklepu
    def run(self):

        print(f"Klient {self.id} przybył do sklepu.")
        time.sleep(random.uniform(1, 5)) # Czas dołączenia klienta do kolejki
        checkout_line = min(checkout_lines, key=lambda x: x.qsize())
        checkout_line.put(self.id)
        print(f"Klient {self.id} dołącza do kolejki {checkout_lines.index(checkout_line)}.")

# Klasa kas
class Checkout(threading.Thread):
    def __init__(self, index):
        threading.Thread.__init__(self)
        self.index = index
        self.customers_served = 0

    # Obsługa klienta
    def run(self):
        while True:
            try:
                customer_id = checkout_lines[self.index].get(timeout=10)
                print(f"Kasa {self.index} obsługuje klienta {customer_id}.")
                time.sleep(random.uniform(5, 10))  # Czas obsługi klienta przez kasę
                self.customers_served += 1
                print(f"Kasa {self.index} zakończyła obsługę klienta {customer_id}.")
            except queue.Empty:
                break

# Symulacja kolejki sklepowej
def simulate_checkout():
    iter = 0

    for i in range(5):
        Checkout(i).start()

    while True:
        Customer(iter).start()
        iter += 1
        time.sleep(random.uniform(1, 2)) # Czas przybycia klienta do sklepu

checkout_lines = [queue.Queue() for _ in range(5)]

simulate_checkout()
